In [6]:
import pandas as pd
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load data from CSV file
with open('buildsfinal2.csv', 'r') as file:
    csvreader = csv.reader(file)
    cpus = []
    motherboards = []
    rams = []
    gpus = []
    psus = []
    for row in csvreader:
        cpus.append(row[1])
        motherboards.append(row[2])
        rams.append(row[3])
        gpus.append(row[4])
        psus.append(row[5])
cpus.pop(0)    
motherboards.pop(0)
rams.pop(0)
gpus.pop(0)
psus.pop(0)
    

data = {
    'CPU': cpus,
    'Motherboard': motherboards,
    'GraphicCard': gpus,
    'Ram': rams,
    'PSU': psus
}


df = pd.DataFrame(data)


df['text'] = df.apply(lambda x: ' '.join(x), axis=1)


tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['text'])


cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)


def recommend_items_for_cpu(input_cpu):
    idx = df.index[df['CPU'] == input_cpu].tolist()[0]
    sim_mb_scores = list(enumerate(cosine_sim[idx]))
    sim_mb_scores = sorted(sim_mb_scores, key=lambda x: x[1], reverse=True)
    sim_mb_scores = sim_mb_scores[1:]  # Exclude the input CPU itself
    recommended_motherboards = [(df.iloc[i[0]]['Motherboard'], i[1]) for i in sim_mb_scores]

    return recommended_motherboards


def recommend_items_for_cpu_and_motherboard(input_cpu, input_motherboard):
    cpu_idx = df.index[df['CPU'] == input_cpu].tolist()[0]
    mb_idx = df.index[df['Motherboard'] == input_motherboard].tolist()[0]

    sim_gc_scores = cosine_sim[cpu_idx, mb_idx]
    recommended_graphic_cards = [(df.iloc[cpu_idx]['GraphicCard'], sim_gc_scores)]

    return recommended_graphic_cards

def recommend_items_for_cpu_and_motherboard_and_gpu(input_cpu, input_motherboard, input_gpu):
    cpu_idx = df.index[df['CPU'] == input_cpu].tolist()[0]
    mb_idx = df.index[df['Motherboard'] == input_motherboard].tolist()[0]
    gpu_idx = df.index[df['GraphicCard'] == input_gpu].tolist()[0]

    sim_ram_scores = cosine_sim[cpu_idx, gpu_idx]
    recommended_ram = [(df.iloc[cpu_idx]['Ram'], sim_ram_scores)]

    return recommended_ram

def recommend_items_for_cpu_and_motherboard_and_gpu_and_ram(input_cpu, input_motherboard, input_gpu, input_ram):
    cpu_idx = df.index[df['CPU'] == input_cpu].tolist()[0]
    mb_idx = df.index[df['Motherboard'] == input_motherboard].tolist()[0]
    gpu_idx = df.index[df['GraphicCard'] == input_gpu].tolist()[0]
    ram_idx = df.index[df['Ram'] == input_ram].tolist()[0]

    sim_psu_scores = cosine_sim[cpu_idx, gpu_idx]
    recommended_psu = [(df.iloc[cpu_idx]['PSU'], sim_psu_scores)]

    return recommended_psu

# Example:
input_cpu = 'AMD Ryzen 9 7950X'
recommended_motherboards = recommend_items_for_cpu(input_cpu)
if recommended_motherboards:
    print("Recommended Motherboard for", input_cpu, "CPU:")
    print(recommended_motherboards[0][0])

    input_motherboard = recommended_motherboards[0][0]
    recommended_graphic_cards = recommend_items_for_cpu_and_motherboard(input_cpu, input_motherboard)
    print("Recommended Graphic Card for", input_cpu, "CPU and", input_motherboard, "motherboard:")
    print(recommended_graphic_cards[0][0])

    input_graphic_card = recommended_graphic_cards[0][0]
    recommended_ram = recommend_items_for_cpu_and_motherboard_and_gpu(input_cpu, input_motherboard, input_graphic_card)
    print("Recommended Ram for", input_cpu, "CPU and", input_motherboard, "motherboard and", input_graphic_card, "graphic card:")
    print(recommended_ram[0][0])

    input_ram = recommended_ram[0][0]
    recommended_psu = recommend_items_for_cpu_and_motherboard_and_gpu_and_ram(input_cpu, input_motherboard, input_graphic_card, input_ram)
    print("Recommended PSU for", input_cpu, "CPU and", input_motherboard, "motherboard and", input_graphic_card, "graphic card and", input_ram, "ram:")
    print(recommended_psu[0][0])
else:
    print("No recommendations found for", input_cpu, "CPU.")

print("\nComplete Build -","\nCPU: ",input_cpu,"\nMotherboard: ",input_motherboard,"\nGraphic Card: ",input_graphic_card,"\nRam: ",input_ram,"\nPSU: ",recommended_psu[0][0],"\n")



Recommended Motherboard for AMD Ryzen 9 7950X CPU:
B650E-F
Recommended Graphic Card for AMD Ryzen 9 7950X CPU and B650E-F motherboard:
RTX 4090
Recommended Ram for AMD Ryzen 9 7950X CPU and B650E-F motherboard and RTX 4090 graphic card:
32 GB (2 x 16 GB) DDR5-6000
Recommended PSU for AMD Ryzen 9 7950X CPU and B650E-F motherboard and RTX 4090 graphic card and 32 GB (2 x 16 GB) DDR5-6000 ram:
1000 W

Complete Build - 
CPU:  AMD Ryzen 9 7950X 
Motherboard:  B650E-F 
Graphic Card:  RTX 4090 
Ram:  32 GB (2 x 16 GB) DDR5-6000 
PSU:  1000 W 

